In [1]:
#================================================================================
# 신규 SetnecePieceBPETokenzer tokenizer vocab 생성 예제
# => tokenizer 방식중, SetnecePieceBPETokenzer 를 이용하여 새롭게 tokenize vocab 을 생성하는 예제임
# => OpenAI GPT-2에서는 ByteLevelBPETokenizer , kogpt-2에서는 SetnecePieceBPETokenzer 를 사용함.
#
# 참고자료 
# https://gist.github.com/lovit/e11c57877aae4286ade4c203d6c26a32
# https://discuss.huggingface.co/t/training-sentencepiece-from-scratch/3477/2
#
#================================================================================
# [과정] => Kogpt-2 tokenizer 방식
#
# 1. SentencePieceBPETokenzer 정의 후 훈련 
#     => 훈련시, corpora 목록, vocab_size, min_frequency 등을 설정함
#
# 2. 훈련한 SentencePieceBPETokenzer 를 PreTrainedTokenizerFast 와 연동
#     =>PreTrainedTokenizerFast(tokenizer_object=stokenizer)
#
# 3. PreTrainedTokenizerFast tokenizer 저장
#     => transforer_tokenizer.save_pretrained(OUT_PATH) 
#     => 정상적으로 저장되면, 해당 폴더에 3개 json 파일이 생성됨(tokenizer.json, special_tokens_map.json, tokenizer_config.json)
#================================================================================

import torch
import numpy as np
import pandas as pd
import tokenizers
from tokenizers import (ByteLevelBPETokenizer,
                        CharBPETokenizer,
                        SentencePieceBPETokenizer,
                        BertWordPieceTokenizer)
# 말뭉치 지정
corpus_path = '../../korpora/kowiki_20190620/wiki_20190620_small.txt'


In [2]:
# 1. SetnecePieceBPETokenzer 정의 후 훈련 
stokenizer = SentencePieceBPETokenizer(add_prefix_space=True)

# 훈련
stokenizer.train(
    files = [corpus_path],
    vocab_size = 52000,  # 최대 vocab 계수 
    special_tokens = ["<cls>", "<eos>", "<mask>", "<unk>", "<pad>"],  # speical token 지정
    min_frequency = 5,   # 빈도수 
    show_progress = True,
    #limit_alphabet=10000, 
)

In [ ]:
vocab = stokenizer.get_vocab()
print(f'vcoab 길이:{len(vocab)}')
print(sorted(vocab, key=lambda x: vocab[x]))  # sort 해서 vocab 출력 

In [6]:
# 테스트
sentence = 'hello'
output = stokenizer.encode(sentence)
print('=>idx   : %s'%output.ids)
print('=>tokens: %s'%output.tokens)
print('=>offset: %s'%output.offsets)
print('=>decode: %s\n'%stokenizer.decode(output.ids))

=>idx   : [109, 75, 8968, 81]
=>tokens: ['▁', 'h', 'ell', 'o']
=>offset: [(0, 1), (0, 1), (1, 4), (4, 5)]
=>decode: hello



In [5]:
# 2. 훈련한 SetnecePieceBPETokenzer 를 PreTrainedTokenizerFast 와 연동
from transformers import PreTrainedTokenizerFast
transforer_tokenizer = PreTrainedTokenizerFast(tokenizer_object=stokenizer)

In [6]:
# 테스트
sentence = '나는 오늘 아침밥을 먹었다.'
output = transforer_tokenizer.encode(sentence)
print(output)
decode = transforer_tokenizer.decode(output)
print(decode)

[9571, 2448, 1050, 6670, 5255, 647, 1554]
나는 오늘 아침을 먹었다.


In [7]:
# PreTrainedTokenizerFast tokenizer 저장
import os
OUT_PATH = './mytoken'
os.makedirs(OUT_PATH, exist_ok=True)
transforer_tokenizer.save_pretrained(OUT_PATH)

('./mytoken/tokenizer_config.json',
 './mytoken/special_tokens_map.json',
 './mytoken/tokenizer.json')

In [16]:
from transformers import GPT2TokenizerFast
#OUT_PATH = 'gpt-2'
# 저장된 tokenzier 불러와서 확인
tokenizer = GPT2TokenizerFast.from_pretrained(OUT_PATH,
                                            bos_token='<cls>',
                                            eos_token='<eos>',
                                            unk_token='<unk>',
                                            pad_token='<pad>',
                                            mask_token='<mask>')

# 테스트
sentence = '나는 오늘 아침밥을 먹었다.'
output = tokenizer.encode(sentence)
print(output)
decode = tokenizer.decode(output)
print(decode)

file gpt-2/config.json not found
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
file gpt-2/config.json not found
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[167, 224, 246, 167, 232, 242, 23821, 246, 97, 167, 232, 246, 23821, 243, 226, 168, 117, 101, 167, 108, 98, 35975, 226, 31619, 101, 117, 168, 245, 230, 46695, 97, 13]
나는 오늘 아침밥을 먹었다.


In [21]:
tokenizer.decode([23821, 246, 97, 167])

' 오�'